In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import csv
import requests
from zipfile import ZipFile
import os
from datetime import date, datetime

In [2]:
from arcgis.gis import GIS
from pathlib import Path
from zipfile import ZipFile

In [3]:
yearDate = date.today().strftime('%m%d%y')
print(str(datetime.now()) + " Running 1_COVID_Daily_Cases_Process")
print(yearDate)

2020-11-16 12:22:34.023721 Running 1_COVID_Daily_Cases_Process
111620


In [4]:
daily_covid_county_service_id = 'a7887f1940b34bf5a02c6f7f27a5cb2c'
gis = GIS()
data_cases = gis.content.get(daily_covid_county_service_id)
data_cases

<Item title:"Florida COVID19 Cases by County" type:Feature Layer Collection owner:Epidemiology.GIS>

In [5]:
query = data_cases.layers[0].query(where="1=1")
df_covid_cases = query.sdf
df_covid_cases.head()

,OBJECTID,OBJECTID_12_13,DEPCODE,COUNTY,COUNTYNAME,County_1,State,PUIsTotal,Age_0_4,Age_5_14,...,MonNow,Shape__Area,Shape__Length,GlobalID,NewPos,NewNeg,NewTested,NewPercPos,MedianAge,SHAPE
0,1,1,21,041,GILCHRIST,Gilchrist,FL,1495,39,86,...,758,9.204908e+08,148547.348006,cb6285a2-da2e-4583-aad5-1221776836ae,19,131,150,0.126667,41,"{""rings"": [[[529493.625602742, 647744.81243718..."
1,2,2,54,107,PUTNAM,Putnam,FL,6295,174,566,...,1612,2.142439e+09,232574.925675,ce511c5d-2638-44a3-b2f8-b13847543637,9,226,235,0.038298,40,"{""rings"": [[[633424.87518893, 650359.936015058..."
2,3,3,62,123,TAYLOR,Taylor,FL,2823,66,168,...,1562,2.685253e+09,267348.178950,541bd5a3-4453-499a-9896-f0edc5353d0b,4,49,53,0.075472,40,"{""rings"": [[[425905.96838189, 699605.625594474..."
3,4,4,46,091,OKALOOSA,Okaloosa,FL,16101,338,1173,...,10691,2.369223e+09,330553.160278,eb5d2450-a892-40fd-9ba4-9821a7438525,106,977,1083,0.097876,40,"{""rings"": [[[170978.169305377, 740209.62491612..."
4,5,5,7,013,CALHOUN,Calhoun,FL,1567,33,120,...,1120,1.487627e+09,192207.890331,3b55aa95-5170-4caf-82a3-e99ad7459349,5,125,130,0.038462,44,"{""rings"": [[[310646.592981238, 733517.62524545..."


In [6]:
df_covid_cases_1 = df_covid_cases.drop(columns=['SHAPE'])
df_Case = df_covid_cases_1.sort_values(by=['DEPCODE'])
df_Case.head()

,OBJECTID,OBJECTID_12_13,DEPCODE,COUNTY,COUNTYNAME,County_1,State,PUIsTotal,Age_0_4,Age_5_14,...,EverMon,MonNow,Shape__Area,Shape__Length,GlobalID,NewPos,NewNeg,NewTested,NewPercPos,MedianAge
65,66,66,1,001,ALACHUA,Alachua,FL,26643,416,691,...,1770,1747,2.510756e+09,272135.471452,9eddb8fb-00dd-45f1-a09d-8e689049e28a,86,1855,1941,0.044307,24
16,17,17,2,003,BAKER,Baker,FL,3163,71,154,...,2226,2225,1.520246e+09,183737.354704,393e3bc9-079e-4ad6-9162-d77ee72feb59,8,120,128,0.062500,41
14,15,15,3,005,BAY,Bay,FL,11176,139,340,...,2044,2037,1.930771e+09,558412.990826,a66f3b06-8ee0-4dec-843e-f41d712aad2f,126,947,1073,0.117428,42
39,40,40,4,007,BRADFORD,Bradford,FL,2977,70,161,...,1899,1868,7.767446e+08,167178.838253,588f1d06-056b-4af5-a2e7-a5366c833ba5,8,97,105,0.076190,46
41,42,42,5,009,BREVARD,Brevard,FL,32581,422,1006,...,3251,3193,2.687644e+09,793218.333609,943db4a8-472c-444e-b81a-0e7cf9758ca2,140,2215,2355,0.059448,44


In [7]:
cases_by_population_id = '09e0d01d8e8e46b6ae678c256d918178'
gis = GIS()
data_popu = gis.content.get(cases_by_population_id)
data_popu

<Item title:"Case Maps for COVID19" type:Feature Layer Collection owner:Epidemiology.GIS>

In [8]:
query_popu = data_popu.layers[0].query(where="1=1")
df_cases_popu = query_popu.sdf
df_cases_popu.head()

,FID,NAME,GEOID,Pop,Pop_MOE,CssYstr,Css7Dys,Css14Dy,CassAll,FID_1,CovidRate,Shape__Area,Shape__Length,SHAPE
0,1,Alachua,12001,263148,0,86,592,989,11632,1,4420,2.509300e+09,262028.034556,"{""rings"": [[[529453.427928006, 647729.16620529..."
1,2,Baker,12003,27785,0,8,51,95,1893,2,6813,1.524801e+09,177464.398185,"{""rings"": [[[547611.939311098, 731650.13314648..."
2,3,Bay,12005,182482,0,126,475,697,7382,3,4045,2.285559e+09,233326.432744,"{""rings"": [[[208213.290884909, 697920.20090373..."
3,4,Bradford,12007,26979,0,8,37,62,1302,4,4826,7.782856e+08,161130.117599,"{""rings"": [[[552463.851341891, 658321.23030116..."
4,5,Brevard,12009,576808,0,140,903,1733,13514,5,2343,3.353781e+09,317459.583010,"{""rings"": [[[694017.58758627, 515464.701569436..."


In [9]:
df_cases_popu_1 = df_cases_popu.drop(columns=['SHAPE'])
df_PP = df_cases_popu_1.sort_values(by=['FID'])
df_PP.head()

,FID,NAME,GEOID,Pop,Pop_MOE,CssYstr,Css7Dys,Css14Dy,CassAll,FID_1,CovidRate,Shape__Area,Shape__Length
0,1,Alachua,12001,263148,0,86,592,989,11632,1,4420,2.509300e+09,262028.034556
1,2,Baker,12003,27785,0,8,51,95,1893,2,6813,1.524801e+09,177464.398185
2,3,Bay,12005,182482,0,126,475,697,7382,3,4045,2.285559e+09,233326.432744
3,4,Bradford,12007,26979,0,8,37,62,1302,4,4826,7.782856e+08,161130.117599
4,5,Brevard,12009,576808,0,140,903,1733,13514,5,2343,3.353781e+09,317459.583010


In [10]:
print("Saving Florida_COVID19_Cases_by_County{}.xlsx in data/s".format(yearDate))
with pd.ExcelWriter('data/Florida_COVID19_Cases_by_County{}.xlsx'.format(yearDate)) as writer:  
    df_Case.to_excel(writer, sheet_name='Florida_COVID19_Cases_by_County')
    df_PP.to_excel(writer, sheet_name='Sheet1')

Saving Florida_COVID19_Cases_by_County111620.xlsx in data/s


In [11]:
df_case = df_Case[['OBJECTID','COUNTYNAME','County_1','State','NewPos','NewTested','PUIsTotal','CasesAll']].copy()
df_case.head()

,OBJECTID,COUNTYNAME,County_1,State,NewPos,NewTested,PUIsTotal,CasesAll
65,66,ALACHUA,Alachua,FL,86,1941,26643,11740
16,17,BAKER,Baker,FL,8,128,3163,1901
14,15,BAY,Bay,FL,126,1073,11176,7635
39,40,BRADFORD,Bradford,FL,8,105,2977,1304
41,42,BREVARD,Brevard,FL,140,2355,32581,13602


In [12]:
a = set(df_PP['NAME'].values)
b = set(df_case['County_1'].values)
a.difference(b)

{'A State'}

In [13]:
b.difference(a)

{'State'}

In [14]:
df_PP.loc[df_PP['NAME']=='A State','NAME']='State'

In [15]:
df_merge = pd.merge(df_PP, df_case, left_on=['NAME'],right_on= ['County_1'], how = 'outer')
df_merge.head(len(df_merge))

,FID,NAME,GEOID,Pop,Pop_MOE,CssYstr,Css7Dys,Css14Dy,CassAll,FID_1,...,Shape__Area,Shape__Length,OBJECTID,COUNTYNAME,County_1,State,NewPos,NewTested,PUIsTotal,CasesAll
0,1,Alachua,12001,263148,0,86,592,989,11632,1,...,2.509300e+09,262028.034556,66,ALACHUA,Alachua,FL,86,1941,26643,11740
1,2,Baker,12003,27785,0,8,51,95,1893,2,...,1.524801e+09,177464.398185,17,BAKER,Baker,FL,8,128,3163,1901
2,3,Bay,12005,182482,0,126,475,697,7382,3,...,2.285559e+09,233326.432744,15,BAY,Bay,FL,126,1073,11176,7635
3,4,Bradford,12007,26979,0,8,37,62,1302,4,...,7.782856e+08,161130.117599,40,BRADFORD,Bradford,FL,8,105,2977,1304
4,5,Brevard,12009,576808,0,140,903,1733,13514,5,...,3.353781e+09,317459.583010,42,BREVARD,Brevard,FL,140,2355,32581,13602
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,65,Wakulla,12129,31877,0,24,86,128,1463,65,...,1.633086e+09,234986.413040,41,WAKULLA,Wakulla,FL,24,317,3405,1472
65,66,Walton,12131,65858,0,37,184,655,2935,66,...,2.973057e+09,274860.715452,51,WALTON,Walton,FL,37,394,5275,3248
66,67,Washington,12133,24566,0,25,80,126,1369,67,...,1.595344e+09,237797.047191,53,WASHINGTON,Washington,FL,25,91,2420,1383
67,68,Unknown,99999,0,0,0,6,11,154,68,...,NaN,NaN,69,Unknown,Unknown,FL,0,522,3217,2096


In [16]:
df_merge1 =df_merge.rename(columns={"NAME": "County"})
df_merge_extract = df_merge1.drop(columns=['OBJECTID','FID_1', 'County_1','Shape__Area','Shape__Length'])
df_merge_extract.head(len(df_merge_extract))

,FID,County,GEOID,Pop,Pop_MOE,CssYstr,Css7Dys,Css14Dy,CassAll,CovidRate,COUNTYNAME,State,NewPos,NewTested,PUIsTotal,CasesAll
0,1,Alachua,12001,263148,0,86,592,989,11632,4420,ALACHUA,FL,86,1941,26643,11740
1,2,Baker,12003,27785,0,8,51,95,1893,6813,BAKER,FL,8,128,3163,1901
2,3,Bay,12005,182482,0,126,475,697,7382,4045,BAY,FL,126,1073,11176,7635
3,4,Bradford,12007,26979,0,8,37,62,1302,4826,BRADFORD,FL,8,105,2977,1304
4,5,Brevard,12009,576808,0,140,903,1733,13514,2343,BREVARD,FL,140,2355,32581,13602
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,65,Wakulla,12129,31877,0,24,86,128,1463,4590,WAKULLA,FL,24,317,3405,1472
65,66,Walton,12131,65858,0,37,184,655,2935,4457,WALTON,FL,37,394,5275,3248
66,67,Washington,12133,24566,0,25,80,126,1369,5573,WASHINGTON,FL,25,91,2420,1383
67,68,Unknown,99999,0,0,0,6,11,154,0,Unknown,FL,0,522,3217,2096


In [17]:
df_merge_extract['P100K_Y'] = df_merge_extract['NewPos'] / df_merge_extract['Pop']*100000
df_merge_extract['P100K_7'] = df_merge_extract['Css7Dys'] / df_merge_extract['Pop']*100000
df_merge_extract['P100K_14'] = df_merge_extract['Css14Dy'] / df_merge_extract['Pop']*100000
df_merge_extract['T100K_Y'] = df_merge_extract['NewTested'] / df_merge_extract['Pop']*100000
df_merge_extract['PRate'] = df_merge_extract['NewPos'] / df_merge_extract['NewTested']
df_merge_extract['TestGoal'] = df_merge_extract['Css14Dy'] / 14 * 30

In [18]:
df_merge_drop_unknown = df_merge_extract.drop([67])
df_merge_drop_unknown

,FID,County,GEOID,Pop,Pop_MOE,CssYstr,Css7Dys,Css14Dy,CassAll,CovidRate,...,NewPos,NewTested,PUIsTotal,CasesAll,P100K_Y,P100K_7,P100K_14,T100K_Y,PRate,TestGoal
0,1,Alachua,12001,263148,0,86,592,989,11632,4420,...,86,1941,26643,11740,32.681229,224.968459,375.834131,737.607734,0.044307,2119.285714
1,2,Baker,12003,27785,0,8,51,95,1893,6813,...,8,128,3163,1901,28.792514,183.552276,341.911103,460.680223,0.062500,203.571429
2,3,Bay,12005,182482,0,126,475,697,7382,4045,...,126,1073,11176,7635,69.047906,260.299646,381.955481,588.003200,0.117428,1493.571429
3,4,Bradford,12007,26979,0,8,37,62,1302,4826,...,8,105,2977,1304,29.652693,137.143704,229.808369,389.191593,0.076190,132.857143
4,5,Brevard,12009,576808,0,140,903,1733,13514,2343,...,140,2355,32581,13602,24.271508,156.551227,300.446596,408.281439,0.059448,3713.571429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,64,Volusia,12127,527634,0,86,580,1229,14127,2677,...,86,1578,35195,14238,16.299177,109.924683,232.926612,299.070947,0.054499,2633.571429
64,65,Wakulla,12129,31877,0,24,86,128,1463,4590,...,24,317,3405,1472,75.289394,269.786994,401.543433,994.447407,0.075710,274.285714
65,66,Walton,12131,65858,0,37,184,655,2935,4457,...,37,394,5275,3248,56.181481,279.388988,994.564062,598.256856,0.093909,1403.571429
66,67,Washington,12133,24566,0,25,80,126,1369,5573,...,25,91,2420,1383,101.766669,325.653342,512.904014,370.430677,0.274725,270.000000


In [19]:
df_merge_drop_unknown['P100K_Y'] =df_merge_drop_unknown['P100K_Y'].astype(int)
df_merge_drop_unknown['P100K_7'] =df_merge_drop_unknown['P100K_7'].astype(int)
df_merge_drop_unknown['P100K_14'] =df_merge_drop_unknown['P100K_14'].astype(int)
df_merge_drop_unknown['T100K_Y'] =df_merge_drop_unknown['T100K_Y'].astype(int)
df_merge_drop_unknown['TestGoal'] =df_merge_drop_unknown['TestGoal'].astype(int)

In [20]:
df_merge_drop_unknown.head()

,FID,County,GEOID,Pop,Pop_MOE,CssYstr,Css7Dys,Css14Dy,CassAll,CovidRate,...,NewPos,NewTested,PUIsTotal,CasesAll,P100K_Y,P100K_7,P100K_14,T100K_Y,PRate,TestGoal
0,1,Alachua,12001,263148,0,86,592,989,11632,4420,...,86,1941,26643,11740,32,224,375,737,0.044307,2119
1,2,Baker,12003,27785,0,8,51,95,1893,6813,...,8,128,3163,1901,28,183,341,460,0.062500,203
2,3,Bay,12005,182482,0,126,475,697,7382,4045,...,126,1073,11176,7635,69,260,381,588,0.117428,1493
3,4,Bradford,12007,26979,0,8,37,62,1302,4826,...,8,105,2977,1304,29,137,229,389,0.076190,132
4,5,Brevard,12009,576808,0,140,903,1733,13514,2343,...,140,2355,32581,13602,24,156,300,408,0.059448,3713


In [21]:
gdf_county_polygon = gpd.read_file('shp/Florida_Counties.shp')
gdf_county_polygon.head()

,OBJECTID,DEPCODE,COUNTY,COUNTYNAME,DATESTAMP,ShapeSTAre,ShapeSTLen,geometry
0,1,21,041,Gilchrist,2000-05-16,9.908353e+09,4.873000e+05,"POLYGON ((-82.65814 29.83028, -82.65803 29.822..."
1,2,54,107,Putnam,2000-05-16,2.305869e+10,7.629677e+05,"POLYGON ((-81.58084 29.83956, -81.57893 29.835..."
2,3,62,123,Taylor,2000-05-16,2.891747e+10,8.772527e+05,"MULTIPOLYGON (((-83.73037 30.30358, -83.70368 ..."
3,4,46,091,Okaloosa,2000-05-16,2.562159e+10,1.087058e+06,"MULTIPOLYGON (((-86.39159 30.64970, -86.39175 ..."
4,5,7,013,Calhoun,2000-05-16,1.604809e+10,6.313440e+05,"POLYGON ((-84.93266 30.60637, -84.93287 30.605..."


In [22]:
a = set(df_merge_drop_unknown['County'].values)
b = set(gdf_county_polygon['COUNTYNAME'].values)
a.difference(b)

{'State'}

In [23]:
b.difference(a)

set()

In [24]:
df_merge_poly = pd.merge(df_merge_drop_unknown, gdf_county_polygon, left_on = ['County'],right_on= ['COUNTYNAME'], how = 'outer')
df_merge_poly.head()

,FID,County,GEOID,Pop,Pop_MOE,CssYstr,Css7Dys,Css14Dy,CassAll,CovidRate,...,PRate,TestGoal,OBJECTID,DEPCODE,COUNTY,COUNTYNAME_y,DATESTAMP,ShapeSTAre,ShapeSTLen,geometry
0,1,Alachua,12001,263148,0,86,592,989,11632,4420,...,0.044307,2119,66.0,1.0,001,Alachua,2000-05-16,2.702324e+10,8.927266e+05,"POLYGON ((-82.41869 29.92298, -82.41825 29.922..."
1,2,Baker,12003,27785,0,8,51,95,1893,6813,...,0.062500,203,17.0,2.0,003,Baker,2000-05-16,1.636216e+10,6.027173e+05,"POLYGON ((-82.05012 30.36233, -82.04949 30.316..."
2,3,Bay,12005,182482,0,126,475,697,7382,4045,...,0.117428,1493,15.0,3.0,005,Bay,2000-05-16,2.084232e+10,1.834701e+06,"MULTIPOLYGON (((-85.38551 30.50018, -85.38650 ..."
3,4,Bradford,12007,26979,0,8,37,62,1302,4826,...,0.076190,132,40.0,4.0,007,Bradford,2000-05-16,8.359863e+09,5.483267e+05,"POLYGON ((-82.04939 30.14322, -82.04938 30.143..."
4,5,Brevard,12009,576808,0,140,903,1733,13514,2343,...,0.059448,3713,42.0,5.0,009,Brevard,2000-05-16,2.893695e+10,2.600567e+06,"MULTIPOLYGON (((-80.80666 28.61413, -80.80790 ..."


In [25]:
df_merge_poly1 = df_merge_poly.drop(columns=['DEPCODE','OBJECTID','COUNTYNAME_y','COUNTY','DATESTAMP','ShapeSTAre','ShapeSTLen'])
df_merge_poly2 = df_merge_poly1.rename(columns={"COUNTYNAME_x": "COUNTYNAME"})
df_merge_poly2

,FID,County,GEOID,Pop,Pop_MOE,CssYstr,Css7Dys,Css14Dy,CassAll,CovidRate,...,NewTested,PUIsTotal,CasesAll,P100K_Y,P100K_7,P100K_14,T100K_Y,PRate,TestGoal,geometry
0,1,Alachua,12001,263148,0,86,592,989,11632,4420,...,1941,26643,11740,32,224,375,737,0.044307,2119,"POLYGON ((-82.41869 29.92298, -82.41825 29.922..."
1,2,Baker,12003,27785,0,8,51,95,1893,6813,...,128,3163,1901,28,183,341,460,0.062500,203,"POLYGON ((-82.05012 30.36233, -82.04949 30.316..."
2,3,Bay,12005,182482,0,126,475,697,7382,4045,...,1073,11176,7635,69,260,381,588,0.117428,1493,"MULTIPOLYGON (((-85.38551 30.50018, -85.38650 ..."
3,4,Bradford,12007,26979,0,8,37,62,1302,4826,...,105,2977,1304,29,137,229,389,0.076190,132,"POLYGON ((-82.04939 30.14322, -82.04938 30.143..."
4,5,Brevard,12009,576808,0,140,903,1733,13514,2343,...,2355,32581,13602,24,156,300,408,0.059448,3713,"MULTIPOLYGON (((-80.80666 28.61413, -80.80790 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,64,Volusia,12127,527634,0,86,580,1229,14127,2677,...,1578,35195,14238,16,109,232,299,0.054499,2633,"MULTIPOLYGON (((-81.09705 29.35180, -81.09292 ..."
64,65,Wakulla,12129,31877,0,24,86,128,1463,4590,...,317,3405,1472,75,269,401,994,0.075710,274,"MULTIPOLYGON (((-84.41197 30.30244, -84.37858 ..."
65,66,Walton,12131,65858,0,37,184,655,2935,4457,...,394,5275,3248,56,279,994,598,0.093909,1403,"POLYGON ((-86.03523 30.98825, -86.03532 30.982..."
66,67,Washington,12133,24566,0,25,80,126,1369,5573,...,91,2420,1383,101,325,512,370,0.274725,270,"POLYGON ((-85.43543 30.56727, -85.46843 30.567..."


In [26]:
result_folder = 'results/{}'.format(yearDate)
if not os.path.exists(result_folder):
    os.makedirs(result_folder)

In [27]:
shpfile_name = "covid_impact_poly"+ yearDate
print("Saving {}.shp in results/".format(shpfile_name))
gdf_merge_poly = gpd.GeoDataFrame(df_merge_poly2, geometry='geometry',crs="EPSG:4326")
gdf_merge_poly.to_file(os.path.join(result_folder,"{}.shp".format(shpfile_name)))

Saving covid_impact_poly111620.shp in results/


In [28]:
print("Saving {}.zip in publish/".format(shpfile_name))
with ZipFile(os.path.join('publish', '{}.zip'.format(shpfile_name)), 'w') as zipObj:
    for file in os.listdir(result_folder):
        if shpfile_name in file:
            file_path = os.path.join(result_folder, file)
            print(file)
            try:
                zipObj.write(file_path, os.path.basename(file_path))
            except OSError as err:
                print("OS error: {0}".format(err))   

Saving covid_impact_poly111620.zip in publish/
covid_impact_poly111620.prj
covid_impact_poly111620.dbf
covid_impact_poly111620.shp
covid_impact_poly111620.cpg
covid_impact_poly111620.shx


In [29]:
print(str(datetime.now()) + " Done with 1_COVID_Daily_Cases_Process")

2020-11-16 12:22:39.539321 Done with 1_COVID_Daily_Cases_Process
